In [1]:
import numpy as np

class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

def y(x):
    return 2**x

def find_tranformation_matrix(b, E, N, Tmax, stopping_crit = 0):

    M = np.shape(E)[0]
    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))
    R = np.zeros(n, dtype = int)
    
    X = [2**i for i in range(N)][::-1]

    counts = Counter()
    for element in X:
        counts.increment(element, 1)

    # run simulation
    for t in range(n):
        
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            if X[i] != 0:
                
                counts.increment(X[i], -1)

                X[i] = 0

                counts.increment(0, 1)
        else:

            i,j = E[np.random.randint(M)]

            if (X[i] != 0 or X[j] != 0) and X[i] != X[j]:
                    
                new = X[i] | X[j]
                counts.increment(X[i], -1)
                counts.increment(X[j], -1)
                counts.increment(new, 2)
                X[i] = X[j] = new
                
        R[t] = non_zero_distinct_rows = len(counts) - (0 in counts)
        
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2)

        if non_zero_distinct_rows == stopping_crit:
            times = times[:t+1]
            R = R[:t+1]
            break

    return times, R


In [2]:
N = 2000
M = 3*N
Tmax = 500
b = np.linspace(0.00002, 1.5, 10)

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

p_mixing_times = []

for i in range(len(b)):
    t_p_mixing = 0
    ## run 10 trials for each b
    for j in range(10):
        times, R = find_tranformation_matrix(b[i], E, N, Tmax, 1)
        p_mix_index = np.min(np.where(R==1)[0])
        t_p_mixing += times[p_mix_index]
    ## average results over the 10 trials
    p_mixing_times.append(t_p_mixing/10)
    print(t_p_mixing/10)

print(p_mixing_times)


6.691006861800398
49.36818626587314
23.08373281126402
13.706243074950493
10.214604015715746
8.173170935069791
6.663595289350388
5.765946074043737
5.73288370358736
4.959678122078398
[6.691006861800398, 49.36818626587314, 23.08373281126402, 13.706243074950493, 10.214604015715746, 8.173170935069791, 6.663595289350388, 5.765946074043737, 5.73288370358736, 4.959678122078398]


In [5]:
b = np.linspace(0.0002, 0.5, 20)
import networkx as nx
N = 2000
## choose network from network X and change M0 and G
M0 = 30

G = nx.barabasi_albert_graph(N, M0)

## return edge matrix (required for the simulation)
E_barabasi = list(G.edges)

M = np.shape(E_barabasi)[0]

t = 0
p_mixing_times = []

for i in range(len(b)):
    t_p_mixing = 0
    ## run 10 trials for each b
    for j in range(10):
        times, R = find_tranformation_matrix(b[i], E_barabasi, N, Tmax, 1)
        p_mix_index = np.min(np.where(R==1)[0])
        t_p_mixing += times[p_mix_index]
    ## average results over the 10 trials
    p_mixing_times.append(t_p_mixing/10)
    print(t_p_mixing/10)

print(p_mixing_times)


7.003289147540677
18.399303277571214
9.499891151218304
6.912973607257006
5.066492399105094
3.8980328341158113
3.2829142225969945
3.00646499349742
2.636968016916652
2.172760385931982
2.072939214736569
1.9393074839292759
1.6352666732673549
1.5826059405566808
1.4840483772884607
1.408949706665802
1.2242536737046505
1.1936143200940272
1.1126873786339029
1.1025692981451063
[7.003289147540677, 18.399303277571214, 9.499891151218304, 6.912973607257006, 5.066492399105094, 3.8980328341158113, 3.2829142225969945, 3.00646499349742, 2.636968016916652, 2.172760385931982, 2.072939214736569, 1.9393074839292759, 1.6352666732673549, 1.5826059405566808, 1.4840483772884607, 1.408949706665802, 1.2242536737046505, 1.1936143200940272, 1.1126873786339029, 1.1025692981451063]
